In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15
!pip install transformers
!pip install numpy==1.19.5
!pip install 'h5py==2.10.0'
!pip install natsort

In [ ]:
import torch
import transformers as ppb
import tensorflow as tf
import tensorflow_hub as hub
import natsort

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, MaxPooling1D, Dropout, Conv1D, Input, LSTM, SpatialDropout1D, Bidirectional, Conv1D, MaxPooling2D, Layer, concatenate

In [ ]:
import numpy
print(numpy.__version__)

1.19.5


In [ ]:
import h5py
print(h5py.__version__)

2.10.0


#IF the numpy version not equal to 1.19.5 and h5py version not equal to 2.10.0 restart the runtime and start again from cell 2

In [ ]:
from google.colab import auth
auth.authenticate_user()
!curl https://sdk.cloud.google.com | bash

!gcloud init

In [ ]:
validation = True

In [ ]:
!mkdir training_directory
!mkdir validation_directory

In [ ]:
if validation:
    
    #!gsutil -m cp gs://masterthesisbert/embeddings_labeled/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_reddit_500* /content/validation_directory/
    #!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_reddit_500* /content/training_directory/
    !gsutil -m cp gs://masterthesisbert/embeddings_labeled/BERT/MyBERT_wide_full-training-features_3D_labeled_reddit_500_val* /content/validation_directory/
    !gsutil -m cp gs://masterthesisbert/embeddings_right/BERT/MyBERT_wide_full-training-features_3D_labeled_reddit_500_tr* /content/training_directory/
    
else:
    
    #!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_3D_labeled_Alada* /content/training_directory/
    #!gsutil -m cp gs://masterthesisbert/embeddings_right/PsychBERT_con_long/MyBERT_wide_full-training-features_2D_labeled_Alada* /content/training_directory/

In [ ]:
import os
if validation:
    arr = os.listdir("/content/training_directory")
    arr = natsort.natsorted(arr)
    print(arr)
    arrval = os.listdir("/content/validation_directory")
    arrval = natsort.natsorted(arrval)
    print(arrval)


else:
    arr = os.listdir("/content/training_directory")
    arr = natsort.natsorted(arr)
    print(arr)

In [ ]:
# load numpy array from csv file
from numpy import loadtxt
import numpy as np

third_dimension = 768

arrays = []
arrays2 = []

In [ ]:
aladag = False
reddit_500 = True

if aladag:
    length = 10
    length_1 = 1
elif reddit_500:
    length = 5
    length_1 = 2

In [ ]:
if(validation==True):
    for i in range(len(arr[0:length])):
        print(arr[i])
        loaded_arr = np.loadtxt('/content/training_directory'+"/"+arr[i])
        arrays.append(loaded_arr)
        print(i)
    
    for i in range(length_1):
      print(arrval[i])
      loaded_arr2 = np.loadtxt('/content/validation_directory'+"/"+arrval[i])
      arrays2.append(loaded_arr2)
      print(i)
    
else:
    
    for i in range(len(arr[0:length])):
       
        loaded_arr = np.loadtxt('/content/training_directory'+"/"+arr[i])
        arrays.append(loaded_arr)
        print(i)

In [ ]:
threeD = True
if threeD:
    
    concatenated_array = np.array([]).reshape(0, arrays[2].shape[1])

    for array in arrays:

        concatenated_array = np.concatenate([concatenated_array, array], axis=0)

    del arrays

    load_original_arr = concatenated_array.reshape(
        concatenated_array.shape[0], concatenated_array.shape[1] // third_dimension, third_dimension)
    del concatenated_array
    data_train_x = load_original_arr

    
    concatenated_array2 = np.array([]).reshape(0, arrays2[0].shape[1])

    for array in arrays2:

        concatenated_array2 = np.concatenate([concatenated_array2, array], axis=0)

    del arrays2

    load_original_arr2 = concatenated_array2.reshape(
        concatenated_array2.shape[0], concatenated_array2.shape[1] // third_dimension, third_dimension)
    del concatenated_array2
    data_train_x2 = load_original_arr2
    
    
else:

    if(validation==False):

        concatenated_array = np.array([]).reshape(0, 768)
        # Create an array to return to

        for array in arrays:

            concatenated_array = np.concatenate([concatenated_array, array], axis=0)
        data_train_x = concatenated_array
    else: 
        concatenated_array = np.array([]).reshape(0, 768)
        # Create an array to return to

        for array in arrays:

            concatenated_array = np.concatenate([concatenated_array, array], axis=0)
        data_train_x = concatenated_array

        concatenated_array2 = np.array([]).reshape(0, 768)
        # Create an array to return to

        for array in arrays2:

            concatenated_array2 = np.concatenate([concatenated_array2, array], axis=0)
        data_train_x2 = concatenated_array2


In [ ]:
import pandas as pd
import tensorflow as tf
if aladag:  

    !gsutil cp gs://masterthesisbert/Aladag_sample_preprocessed.csv  /content/Aladag_sample_preprocessed.csv
    
    df_name = "/content/Aladag_sample_preprocessed.csv"
    df = pd.read_csv(df_name)

    !gsutil cp gs://masterthesisbert/Aladag_labeled_preprocessed.csv  /content/Aladag_labeled_preprocessed.csv

    df_name2 = "/content/Aladag_labeled_preprocessed.csv"
    df2 = pd.read_csv(df_name2)
    df2

    train_y_tensor = torch.tensor(df[0:10000].binary_annotation.values)
    train_y_tensor2 = torch.tensor(df2[0:10000].binary_annotation.values)

   
    !gsutil cp gs://masterthesisbert/Aladag_labeled_preprocessed_features.csv  /content/Aladag_labeled_preprocessed_features.csv
    !gsutil cp gs://masterthesisbert/Aladag_sample_preprocessed_features.csv  /content/Aladag_sample_preprocessed_features.csv

    df_name2_features = "Aladag_labeled_preprocessed_features.csv"
    df_name_features = "Aladag_sample_preprocessed_features.csv"

    df_features = pd.read_csv(df_name_features)

    df2_features = pd.read_csv(df_name2_features)


    a = ["anxiety", "depression", "control", "ADHD", "bipolar disorder", "autism", "PTSD", "OCD", "schizophrenia", "eating disorder"]
    diseases = sorted(map(lambda x: x.lower(), a))
    diseases

    train_x_tensor_diseases = torch.tensor(df_features[0:10000][diseases].values)
    train_x_tensor2_diseases = torch.tensor(df2_features[0:10000][diseases].values)

    sentiment = ["negative", "neutral", "positive"]

    train_x_tensor_sentiment = torch.tensor(df_features[0:10000][sentiment].values)
    train_x_tensor2_sentiment = torch.tensor(df2_features[0:10000][sentiment].values)

    train_x_tensor_pronouns = torch.tensor(df_features[0:10000][["first","third"]].values)
    train_x_tensor2_pronouns = torch.tensor(df2_features[0:10000][["first","third"]].values)

    train_x_tensor_emotions= torch.tensor(df_features[0:10000][["anger", "fear", "joy", "sadness"]].values)
    train_x_tensor2_emotions = torch.tensor(df2_features[0:10000][["anger", "fear", "joy", "sadness"]].values)

    # all together 
    all_features = diseases + ["negative", "neutral", "positive"] + ["first","third"] + ["anger", "fear", "joy", "sadness"]

    train_x_tensor_features_full = torch.tensor(df_features[0:10000][all_features].values)
    train_x_tensor2_features_full = torch.tensor(df2_features[0:10000][all_features].values)

elif reddit_500:
    !gsutil cp gs://masterthesisbert/reddit_500_final_val.csv  /content/reddit_500_final_val.csv

    df_name2 = "/content/reddit_500_final_val.csv"
    df2 = pd.read_csv(df_name2)
    df2

    !gsutil cp gs://masterthesisbert/reddit_500_final_train.csv  /content/reddit_500_final_train.csv

    df_name = "/content/reddit_500_final_train.csv"
    df = pd.read_csv(df_name)
    df

    train_y_tensor = torch.tensor(df[0:10000].Label.values)
    train_y_tensor2 = torch.tensor(df2[0:10000].Label.values)


    !gsutil cp gs://masterthesisbert/reddit_500_final_val_features.csv  /content/reddit_500_final_val_features.csv
    !gsutil cp gs://masterthesisbert/reddit_500_final_train_features.csv /content/reddit_500_final_train_features.csv

    df_name2_features = "reddit_500_final_val_features.csv"
    df_name_features = "reddit_500_final_train_features.csv"

    df_features = pd.read_csv(df_name_features)

    df2_features = pd.read_csv(df_name2_features)


    a = ["anxiety", "depression", "control", "ADHD", "bipolar disorder", "autism", "PTSD", "OCD", "schizophrenia", "eating disorder"]
    diseases = sorted(map(lambda x: x.lower(), a))
    diseases

    train_x_tensor_diseases = torch.tensor(df_features[0:10000][diseases].values)
    train_x_tensor2_diseases = torch.tensor(df2_features[0:10000][diseases].values)

    sentiment = ["negative", "neutral", "positive"]

    train_x_tensor_sentiment = torch.tensor(df_features[0:10000][sentiment].values)
    train_x_tensor2_sentiment = torch.tensor(df2_features[0:10000][sentiment].values)

    train_x_tensor_pronouns = torch.tensor(df_features[0:10000][["first","third"]].values)
    train_x_tensor2_pronouns = torch.tensor(df2_features[0:10000][["first","third"]].values)

    train_x_tensor_emotions= torch.tensor(df_features[0:10000][["anger", "fear", "joy", "sadness"]].values)
    train_x_tensor2_emotions = torch.tensor(df2_features[0:10000][["anger", "fear", "joy", "sadness"]].values)

    # all together 
    all_features = diseases + ["negative", "neutral", "positive"] + ["first","third"] + ["anger", "fear", "joy", "sadness"]

    train_x_tensor_features_full = torch.tensor(df_features[0:10000][all_features].values)
    train_x_tensor2_features_full = torch.tensor(df2_features[0:10000][all_features].values)

In [ ]:
import numpy as np

from tensorflow import keras
from tensorflow.python.keras.layers import Input, Dense,RepeatVector, TimeDistributed, Dense, Dropout, LSTM, Bidirectional
from tensorflow.python.keras.models import Sequential

import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
print(np.__version__)

1.19.5


In [ ]:
# Conctruct autoencoder for training test to find anomalous days in the data
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
import os
import random as rn 
from tensorflow.compat.v1.keras import backend as K
number = 2
os.environ['PYTHONHASHSEED']=str(number)
#Set random seed for numpy, python and tensorflow
np.random.seed(number)
rn.seed(number)
tf.set_random_seed(number)
# Set the number of threads to 1 
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
# Its an example of an autoencoder. I might use it later. 

In [ ]:
def unison_shuffled_copies(a, b, just_indices=False):
        assert len(a) == len(b)
        p = numpy.random.permutation(len(a))
        return a[p], b[p], p 

In [ ]:
permutation_done = True

if permutation_done and reddit_500:
    !gsutil cp gs://masterthesisbert/parameter_optimization/permutation.csv  /content/permutation.csv
    from numpy import loadtxt
    p = loadtxt('permutation.csv', delimiter=',')
    data_train_x_permut, train_y_tensor_permut = data_train_x[p.astype(int)], train_y_tensor[p.astype(int)]

elif(permutation_done==False and reddit_500==True):
    
    data_train_x_permut, train_y_tensor_permut, p = unison_shuffled_copies(data_train_x, train_y_tensor)

    from numpy import savetxt
    savetxt('permutation.csv', p, delimiter=',')

    from numpy import loadtxt
    p = loadtxt('permutation.csv', delimiter=',')
    p
    #!gsutil cp /content/permutation.csv gs://masterthesisbert/parameter_optimization/permutation.csv

In [ ]:
noise_detection = True
if noise_detection:
    #for the novelty detection the data is already permutated
    !gsutil cp gs://masterthesisbert/anomaly_indices/BERT_reddit_500_noisy1.txt /content/
    #!gsutil cp gs://masterthesisbert/anomaly_indices/PsychBERT1_reddit_500_noisy1.txt /content/

    my_file = open("/content/BERT_reddit_500_noisy1.txt", "r")
    content = my_file.read()
    normal_data = content.split("\n")
    normal_data.pop()

    normal_data = [int(i) for i in normal_data]
    normal_data = sorted(normal_data)

    if reddit_500:
        data_train_x_permut = data_train_x_permut[normal_data]
        train_y_tensor_permut = train_y_tensor_permut[normal_data]
    elif aladag:
        data_train_x = data_train_x[normal_data]
        train_y_tensor = train_y_tensor[normal_data]
    
    train_x_tensor_diseases =  train_x_tensor_diseases[normal_data]

    sentiment = ["negative", "neutral", "positive"]

    train_x_tensor_sentiment = train_x_tensor_sentiment[normal_data]

    train_x_tensor_pronouns = train_x_tensor_pronouns[normal_data]

    train_x_tensor_emotions= train_x_tensor_emotions[normal_data]

    # all together 
    all_features = diseases + ["negative", "neutral", "positive"] + ["first","third"] + ["anger", "fear", "joy", "sadness"]

    train_x_tensor_features_full = train_x_tensor_features_full[normal_data]

In [ ]:
data_train_x = data_train_x_permut
train_y_tensor = train_y_tensor_permut

In [ ]:
def create_bilstm(data_train_x=data_train_x, number_of_classes=2, learn_rate=0.0001, training = True, momentum = 0.2, dropout_rate=0.3):
    # Bi-LSTM
    bilstm = Sequential()

    pool_size = 2

    bilstm.add(Input(shape=(256,768)))
    bilstm.add(Bidirectional(LSTM(20, return_sequences=True, dropout=dropout_rate, recurrent_dropout=0.2)))
    bilstm.add(MaxPooling1D(pool_size = pool_size))
    bilstm.add(Flatten())
    bilstm.add(Dropout(0.5)) 

    return bilstm

In [ ]:
def create_lstm_cnn(data_train_x=data_train_x, training = True, number_of_classes=2, learn_rate=0.001, momentum = 0.2, dropout_rate=0.3):

    lstm_cnn = Sequential() 

    lstm_cnn.add(Input(shape=(256,768)))

    lstm_cnn.add(Dropout(0.5)) 
    lstm_cnn.add(LSTM(units=100, return_sequences=True)) 
    lstm_cnn.add(Conv1D(3, (8,), padding='same', activation='relu')) 
    lstm_cnn.add(MaxPooling1D(2))
    lstm_cnn.add(Flatten())

    
    return lstm_cnn

In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1), initializer="normal") 
        self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1), initializer="zeros")
        super(Attention, self).build(input_shape)


    def call(self,x):

        e = K.tanh(K.dot(x, self.W) +self.b) 
        a = K.softmax(e, axis=1) 
        output = x*a

        if self.return_sequences:
            return output

        return K.sum(output, axis=1)

def create_lstm_atten_cnn(data_train_x=data_train_x, number_of_classes=2, training = True, learn_rate=0.001, momentum = 0.2, dropout_rate=0.3):


    lstm_atten_cnn = Sequential() 

    lstm_atten_cnn.add(Input(shape=(256,768)))

    lstm_atten_cnn.add(Dropout(0.5)) 
    lstm_atten_cnn.add(LSTM(units=100, return_sequences=True)) 
    lstm_atten_cnn.add(Attention(return_sequences=True)) 
    lstm_atten_cnn.add(Conv1D(3, (8,), padding='same', activation='relu')) 
    lstm_atten_cnn.add(MaxPooling1D(2))
    lstm_atten_cnn.add(Flatten()) 

    return lstm_atten_cnn

In [ ]:
# Fully Dense Network

def create_dense(data_train_x=data_train_x, number_of_classes = 2, learn_rate=0.0001, momentum = 0.2, dropout_rate=0.3):

    dense = Sequential()

    dense.add(Input(shape=(768,)))

    dense.add(keras.layers.Dropout(rate=dropout_rate))

    dense.add(Dense(384, activation='relu',  kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

    dense.add(keras.layers.Dropout(rate=dropout_rate))

    dense.add(Dense(128, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

    dense.add(keras.layers.Dropout(rate=dropout_rate))

    return dense

def create_dense_num(data_train_x=train_x_tensor_diseases, number_of_classes = 2, learn_rate=0.0001, momentum = 0.2, dropout_rate=0.3):

    dense_num = Sequential()


    dense_num.add(Input(shape=(data_train_x.shape[1],)))

    dense_num.add(keras.layers.Dropout(rate=dropout_rate))

    dense_num.add(Dense(5, activation='relu',  kernel_initializer=keras.initializers.glorot_uniform(seed=number)))

    dense_num.add(keras.layers.Dropout(rate=dropout_rate))

    
    return dense_num

def concatenated_dense(data_train_x, train_x_tensor_diseases, learn_rate, dropout_rate=0.5, dense_la = True, bi = False, lstm_cnn_la = False, lstm_a_cnn_la=False):


    dense_num = create_dense_num(data_train_x=train_x_tensor_diseases, learn_rate =learn_rate, dropout_rate=0.4)

    # concatenate outputs

    if dense_la:  
        
        dense = create_dense(data_train_x=data_train_x, learn_rate=learn_rate, dropout_rate=0.4)


        combinedInput = concatenate([dense.output, dense_num.output])

    elif bi:

        bilstm = create_bilstm(data_train_x=data_train_x, learn_rate=learn_rate, dropout_rate=0.4)

        combinedInput = concatenate([bilstm.output, dense_num.output])

    elif lstm_cnn_la:

        lstm_cnn = create_lstm_cnn(data_train_x=data_train_x, learn_rate=learn_rate, dropout_rate=0.4)

        combinedInput = concatenate([lstm_cnn.output, dense_num.output])


    elif lstm_a_cnn_la:

        lstm_atten_cnn = create_lstm_atten_cnn(data_train_x=data_train_x, learn_rate=learn_rate, dropout_rate=0.4)

        combinedInput = concatenate([lstm_atten_cnn.output, dense_num.output])


    x = Dense(64, kernel_initializer=keras.initializers.glorot_uniform(seed=66), activation = 'relu')(combinedInput)

    x = Dense(1, kernel_initializer=keras.initializers.glorot_uniform(seed=66), activation="linear")(x)

    # define the strutcure of the input to the model 
    if dense_la:  
        
        model = Model(inputs=[dense.input, dense_num.input], outputs=x)

    elif bi:
        
        model = Model(inputs=[bilstm.input, dense_num.input], outputs=x)

    elif lstm_cnn_la:

        model = Model(inputs=[lstm_cnn.input, dense_num.input], outputs=x)


    elif lstm_a_cnn_la:

        model = Model(inputs=[lstm_atten_cnn.input, dense_num.input], outputs=x)

    adam = keras.optimizers.Adam(lr=1e-3, decay=1e-6)

    optimizer = keras.optimizers.Adam(learning_rate=learn_rate)

    model.compile(loss="mse", optimizer="adam" , metrics=['accuracy'])
    model.summary()

    return model

In [ ]:
def summarize_results(grid_result, model_name, result_to_file = True):
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    if result_to_file == True:
        string_to_save = f"Best: {grid_result.best_score_} using {grid_result.best_params_} \n" 
        for mean, stdev, param in zip(means, stds, params):
            string_to_save = string_to_save + f"{mean} ({stdev}) with: {param} \n" 
        text_results = open(model_name + ".txt", "w")
        n = text_results.write(string_to_save)
        text_results.close()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
# each time the training set is the 4/5 of the normal data or a fixed number 
train_test_split = int(round(len(data_train_x)/5, 0)*4)

#train_test_split = 4000 
train_test_split 

In [ ]:
train_different_manner = True
if train_different_manner:

    learn_rate = [0.0001, 0.00001]

    epochs= [10, 13, 15]

    #dropout_rate =[0.3, 0.4, 0.5]

    for i in range(len(learn_rate)):

        for j in range(len(epochs)):

            #if for dense move the code under the dropout_rate to the right
            #for z in range(len(dropout_rate)):


            final_model = concatenated_dense(data_train_x[:train_test_split], train_x_tensor_pronouns[:train_test_split], learn_rate=learn_rate[i], dense_la=False, bi=True)
            final_model.fit([data_train_x[:train_test_split], train_x_tensor_pronouns[:train_test_split]], train_y_tensor[:train_test_split],  epochs = epochs[j])

            predicted2 = final_model.predict([data_train_x[train_test_split:], train_x_tensor_pronouns[train_test_split:]])

            
            predicted2 = np.where(predicted2<0.5, 0, 1)


            # if dense_la True uncomment it. 
            #if(i==0 and j==0 and z==0):

            #comment it out if dense_la true 
            if(i==0 and j==0):

                model_text = "" + "\n" + "learn_rate: " + str(learn_rate[i]) + " epochs " + str(epochs[j]) + "\n" # + "dropout_rate" + dropout_rate[z]
            else:
                model_text = model_text + "\n" + "learn_rate: " + str(learn_rate[i]) + " epochs " + str(epochs[j]) + "\n"

            #accuracy
            model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5)) 
            print("Accuracy:", round(accuracy_score(train_y_tensor[train_test_split:], predicted2), 5))


            #precision
            model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor[train_test_split:], predicted2), 5)) 

            print("Precision:", round(precision_score(train_y_tensor[train_test_split:], predicted2), 5))
            #recall
            model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor[train_test_split:], predicted2), 5)) 

            print("Recall:", round(recall_score(train_y_tensor[train_test_split:], predicted2), 5))

            #F1Score
            model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 

            print("F1score:", round(f1_score(train_y_tensor[train_test_split:], predicted2), 5)) 


    print(model_text)

    name = "Noise_Results_BERT_BiLSTM_reddit_500_pronouns_notcross.txt"

    with open(name, "w") as text_file:
            text_file.write(model_text)

    !gsutil cp /content/{name} gs://masterthesisbert/parameter_optimization

In [ ]:
# train the model with the whole dataset 
final = concatenated_dense(data_train_x, train_x_tensor_diseases, 0.0001, 0.4, dense_la=False, bi=False, lstm_a_cnn_la=True)
final.fit(x = [data_train_x, train_x_tensor_diseases], y =  train_y_tensor, epochs=10)

In [ ]:
validation = True

In [ ]:
if validation:

    predicted2 = final.predict([data_train_x2,train_x_tensor2_diseases])

    predicted2 = np.where(predicted2<0.5, 0, 1)
    #predicted2 = np.where(predicted2==np.max(predicted2, axis=1), 1, 0)

    model_text = ""

    model_text = model_text + "BERT LSTM-A-CNN diseases"
    print(model_text)
    #accuracy

    model_text = model_text + "\n" + "Accuracy:" + str(round(accuracy_score(train_y_tensor2, predicted2), 5)) 
    
    print("Accuracy:", round(accuracy_score(train_y_tensor2, predicted2), 5))


    #precision
    model_text = model_text + "\n" + "Precision:" + str(round(precision_score(train_y_tensor2, predicted2), 5)) 

    print("Precision:", round(precision_score(train_y_tensor2, predicted2), 5))
    #recall
    model_text = model_text + "\n" + "Recall:" + str(round(recall_score(train_y_tensor2, predicted2), 5)) 

    print("Recall:", round(recall_score(train_y_tensor2, predicted2), 5))

    #F1Score
    model_text = model_text + "\n" + "F1score:" + str(round(f1_score(train_y_tensor2, predicted2), 5)) 

    print("F1score:", round(f1_score(train_y_tensor2, predicted2), 5))